In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
!pip install stemgraphic
import stemgraphic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=eb238e1bd49f6b900935bccb5eb83664c732a153553ba9187827c6e1e12846ff
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
df = pd.read_csv(r'/content/Tanmay_22M1850_data.csv')

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

#Exploratory data analysis

In [ ]:
df.keys()

In [ ]:
parameters = ['Time(s)', 'Extension(mm)', 'Load(N)', 'Compressive extension(mm)',
       'Compressive strain (Extension)(mm/mm)', 'Compressive stress(MPa)']

In [ ]:
plt.figure(figsize= (6,5))
for i in parameters:
    sns.histplot(x= i,data= df)
    plt.title(f'histogram of {i}')
    plt.figure()

In [ ]:
plt.figure(figsize= (8,6))
for i in parameters:
    plt.figure()
    sns.boxplot(y= i,data= df)
    plt.title(f'boxplot of {i}')
    print('\n')

    print(f'Q1, Q2,Q3,Q4 of {i} are {np.quantile(df[i],[0.25,0.5,0.75,1])}')

In [ ]:
plt.figure(figsize= (8,6))
for i in parameters:
    fig,ax = stemgraphic.stem_graphic(df[i],aggregation= False)
    ax.set_title(f'stem & leaf plot of {i}')
    plt.figure()

#Distribution analysis

In [ ]:
from scipy import stats
import statsmodels.graphics.gofplots as sm
#plt.figure(figsize=(6, 4))
for i in parameters:
  fig, ax = plt.subplots(1, 3, figsize=(15, 5))
  fig.subplots_adjust(hspace=0.8, wspace=.3)
  a=sm.ProbPlot(df[i],fit=True,dist=stats.norm,).ppplot(line='45',ax=ax[0])
  ax[0].set_title(f'PP plot -assumed dist(normal)')

  b=sm.ProbPlot(df[i],fit=True,dist=stats.norm).qqplot(line='45',ax=ax[1])
  ax[1].set_title(f'QQ plot- assumed dist(normal)')
  sns.kdeplot(df[i],ax=ax[2])
  ax[2].set_title(f'Density plot {i}- assumed dist(normal)')
  plt.show()
  plt.tight_layout()

In [ ]:
from scipy import stats
import statsmodels.graphics.gofplots as sm

for i in parameters:
  xt,_ = stats.boxcox(df.loc[df[i]>0][i])
  fig, ax = plt.subplots(1, 3, figsize=(15, 5))
  fig.subplots_adjust(hspace=0.8, wspace=.3)
  a=sm.ProbPlot(xt,fit=True,dist=stats.norm,).ppplot(line='45',ax=ax[0])
  ax[0].set_title(f'PP plot -assumed dist(lognormal)')

  b=sm.ProbPlot(xt,fit=True,dist=stats.norm).qqplot(line='45',ax=ax[1])
  ax[1].set_title(f'QQ plot- assumed dist(lognormal)')

  sns.kdeplot(xt,ax=ax[2])
  ax[2].set_title(f'Density plot {i}- assumed dist(lognormal)')
  plt.show()
  plt.tight_layout()

In [ ]:
df

In [ ]:
df.keys()

In [ ]:
sns.pairplot(df)

In [ ]:
sns.heatmap(df.corr(),annot = True)

#linear regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [ ]:
df.keys()

#simple linear regression

In [ ]:
features = ['Time(s)', 'Extension(mm)', 'Load(N)', 'Compressive extension(mm)',
       'Compressive strain (Extension)(mm/mm)']

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

x = df[features]
y=df['Compressive stress(MPa)']
for i in features:
  x=df[i]
  # add a constant term to the features
  x= sm.add_constant(x)
  model = sm.OLS(y, x).fit()
  print(model.summary())

#Regression plot between Target & independent variable.

In [ ]:
for i in features:
  sns.regplot(x=i, y='Compressive stress(MPa)', data=df)
  plt.title(f'Regression plot between {i} and Compressive stress(MPa)')
  lr.fit(df[i].values.reshape(-1, 1),df['Compressive stress(MPa)'].values.reshape(-1, 1))
  print(f'y={lr.intercept_[0]:.2f}+{lr.coef_[0][0]:.2f}x')
  plt.show()

#Multiple Linear ergression

In [ ]:
df.keys()

In [ ]:
df = df.rename(columns={'Time(s)':'Time',
                        'Extension(mm)':'extension',
                        'Load(N)': 'Load',
                        'Compressive extension(mm)': 'compextension',
                        'Compressive strain (Extension)(mm/mm)': 'compstrain',
                        'Compressive stress(MPa)': 'comp_stress',
                        })

In [ ]:
df.head()

In [ ]:
 model = ols(formula='comp_stress~Time+extension+Load+compextension+compstrain',data = df).fit()
 model.summary()

# hypothesis testing

In [ ]:
features = df.keys()

In [ ]:
from scipy.stats import ttest_1samp

# Set significance level
alpha = 0.05
# Create an empty dataframe to store the t-test results
results = pd.DataFrame(columns=['Column', 'Population mean','Sample mean','Sample_std_dev','t-value','p-value','result'])

# Perform hypothesis testing for each column
for col in features:
  # Take 8 random samples of size 100 from the column
  pop_mean=df[col].mean()
  mysample=df[col].sample(n=100)
  sample_mean = np.mean(mysample)
  # Calculate the sample standard deviation
  sample_std = np.std(mysample, ddof=1)
  # Calculate the t-value and p-value using one-sample t-test
  t_value, p_value = ttest_1samp(mysample, popmean=df[col].mean())

  # Determine if the null hypothesis is rejected based on the p-value
  if p_value < alpha:
    result = 'rejected'
  else:
    result = 'not rejected'
  # Add the t-test results to the dataframe
  results = results.append({'Column': col, 'Population mean': pop_mean,'Sample mean': sample_mean,'Sample_std_dev':
                            sample_std ,'t-value': t_value ,'p-value': p_value,'result': result},ignore_index= True)

In [ ]:
results